In [ ]:
using System;
using System.IO;
using System.Linq;
using System.Text.RegularExpressions;

var data = File.ReadAllText("inputs/input_day4.txt");

In [ ]:
var data = 
@"7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7";

In [ ]:
class Board
{
    private List<int> _numbers;
    private List<bool> _marked;

    private List<int> _colIndices;

    private Regex _matcher = new Regex(@"\d+", RegexOptions.Compiled);

    private bool _won = false;

    public Board(string all)
    {
        var match = _matcher.Matches(all);
        _numbers = match.Select(_ => int.Parse(_.Value)).ToList();
        _marked = new List<bool>(
            Enumerable.Range(0, _numbers.Count())
                .Select(_ => false)
        );

        _colIndices = Enumerable.Range(0, 5)
            .SelectMany(col => 
                Enumerable.Range(0, 5)
                    .Select(row => row * 5 + col)
            )
            .ToList();
    }

    public int Score(int drawn)
    {
        var index = _numbers.IndexOf(drawn);

        if(index == -1 || _won)
        {
            return 0;
        }

        _marked[index] = true;

        //Check rows
        var rowWin = Enumerable.Range(0, 5)
            .Select(row => 
                _marked
                    .Skip(row * 5)
                    .Take(5)
                    .All(_ => _)
            )
            .Any(_ => _);

        //Check columns
        var colWin = Enumerable.Range(0, 5)
                .Select(col => 
                    _colIndices
                        .Skip(col * 5)
                        .Take(5)
                        .All(_ => _marked[_])
                )
                .Any(_ => _);

        if(!(rowWin || colWin))
        {
            return 0;
        }

        _won = true;

        return drawn * _numbers
            .Zip(
                _marked,
                (number, marked) => marked ? 0 : number
            )
            .Sum();
    }
}

In [ ]:
int[] drawn;

var boards = new List<Board>();

using(var reader = new StringReader(data))
{
    drawn = reader
        .ReadLine()
        .Split(',')
        .Where(_ => !string.IsNullOrWhiteSpace(_))
        .Select(_ => int.Parse(_))
        .ToArray();

    string boardFromFile = "";

    var lineCounter = 0;

    string line;

    while((line = reader.ReadLine()) != null)
    {
        boardFromFile += line + "\n";
        ++lineCounter;
        
        if(lineCounter % 6 == 0)
        {
            boards.Add(new Board(boardFromFile));
            boardFromFile = "";
        }
    }
}

var winner = drawn
    .SelectMany(
        number => 
            boards.Select(board => board.Score(number))
    )
    .Where(score => score != 0)
    .Last();

Console.WriteLine(winner);

10478
